<a href="https://colab.research.google.com/github/MickaRiv/ProjetDatascientest-VoiceSeparator/blob/main/streamlit/streamlit_voice-separation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prérequis



*   Avoir un compte ngrok sur https://dashboard.ngrok.com
*   récupérer le numéro de token



#Installation de streamlit et ngrok

In [2]:
!pip install streamlit

     |████████████████████████████████| 9.7 MB 5.2 MB/s 
     |████████████████████████████████| 164 kB 68.1 MB/s 
     |████████████████████████████████| 76 kB 4.1 MB/s 
     |████████████████████████████████| 4.3 MB 42.8 MB/s 
     |████████████████████████████████| 181 kB 39.7 MB/s 
     |████████████████████████████████| 111 kB 41.2 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
     |████████████████████████████████| 128 kB 66.7 MB/s 
     |████████████████████████████████| 792 kB 50.1 MB/s 
     |████████████████████████████████| 380 kB 48.9 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=fda892159aaef2c7686e84fb1300d483fc38ed452fdaa4ea81bb0153cc219278
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

In [3]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=484448d47dcb81624332e01417d3884c2f6a439fcc38bf344b8b7cb569748ee9
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


# Le streamlit

## pip install nussl

In [7]:
!pip install nussl

     |████████████████████████████████| 239 kB 5.0 MB/s 
     |████████████████████████████████| 51 kB 53 kB/s 
     |████████████████████████████████| 3.3 MB 41.6 MB/s 
     |████████████████████████████████| 153 kB 58.2 MB/s 
     |████████████████████████████████| 310 kB 48.1 MB/s 
     |████████████████████████████████| 251 kB 53.7 MB/s 
     |████████████████████████████████| 90 kB 7.9 MB/s 
     |████████████████████████████████| 963 kB 45.5 MB/s 
     |████████████████████████████████| 130 kB 62.3 MB/s 
     |████████████████████████████████| 6.2 MB 30.3 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4712 sha256=61135ba57f42009b0f1d2ffb21a98e824eeac1f89d31e111fbccde9132279ddb
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for gpytorch: filename=gpytorch-1.6.0-py2.py3-none-any.whl size=509889 sha256=3c3ab32ea36f25b5ac9d00ff44359ad1322052cdeb0298a5942c20e38cc3af62
  Stored in d

## pip install voicesep

In [27]:
!pip install git+https://github.com/MickaRiv/ProjetDatascientest-VoiceSeparator

  Cloning https://github.com/MickaRiv/ProjetDatascientest-VoiceSeparator to /tmp/pip-req-build-mqyxd3rn
  Running command git clone -q https://github.com/MickaRiv/ProjetDatascientest-VoiceSeparator /tmp/pip-req-build-mqyxd3rn
  Created wheel for voice-separator: filename=voice_separator-0.0.1-py3-none-any.whl size=7761 sha256=32f26ed84ee0fd2a14caff12f17a1ddfceee038878b639a64d8cae9654ee3a02
  Stored in directory: /tmp/pip-ephem-wheel-cache-dwakdrmz/wheels/89/ad/91/9cded351057b1a842fbbfa97f091da426d85036b0efd2e0273
Successfully built voice-separator


## Montage du drive

In [26]:
from google.colab import drive # import drive from google colab
drive.mount("/content/drive")  # we mount the google drive at /content/drive

Mounted at /content/drive


## Développement de st_voice-sepration.py

### récupération de st_voice-sepration.py du git

In [34]:
!wget https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/streamlit/st_voice-separation.py

--2022-02-25 14:04:07--  https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/streamlit/st_voice-separation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6143 (6.0K) [text/plain]
Saving to: ‘st_voice-separation.py’

st_voice-separation 100%[===================>]   6.00K  --.-KB/s    in 0s      

2022-02-25 14:04:07 (42.1 MB/s) - ‘st_voice-separation.py’ saved [6143/6143]



### dev temporaire du .py dans drive pour éviter les commit push à répétition

In [39]:
!ls /content/drive/MyDrive/'Projet Datascientest'/Ephi_st

st_voice-separation.py


### ou dev direct dans colab

In [ ]:
%%writefile st_voice-separation_from_colab.py
import streamlit as st

st.title("st.title")


Overwriting st_voice-separation.py


#Lancement du streamlit

Start ngrock

In [22]:
!ls
!ngrok authtoken 25QRE4uYk7AG52BigElvRGk5TZc_4EfgeeiWXnGXYY2Qf5axB
#!ngrok

sample_data  st_voice-separation.py
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com 

In [23]:
!streamlit run --server.port 80 st_voice-separation.py &>/dev/null& # en background

In [70]:
!streamlit run --server.port 80 /content/drive/MyDrive/'Projet Datascientest'/Ephi_st/st_voice-separation.py  &>/dev/null&

In [71]:
from pyngrok import ngrok
# Setup u tunnel to the streamlit port 8501
public_url = ngrok.connect(port='80')
public_url

<NgrokTunnel: "http://82b9-35-196-227-157.ngrok.io" -> "http://localhost:80">

# Arrêt de streamlit

Récupération de l'ID du process

In [67]:
!ps -eaf | grep streamlit

root         523       1  2 12:23 ?        00:05:28 /usr/bin/python3 /usr/local/bin/streamlit run --server.port 80 st_voice-separation.py
root        3119      73  0 16:33 ?        00:00:00 /bin/bash -c ps -eaf | grep streamlit
root        3121    3119  0 16:33 ?        00:00:00 grep streamlit


Kill du process

In [68]:
!kill 523

Arrêt de ngrok

In [69]:
ngrok.kill()